## Training Custom Models with AI Services

This sample trains a custom image classifier to identify cracked and uncracked pavement

![An image of cracked and uncracked pavement](https://learn.microsoft.com/dotnet/machine-learning/tutorials/media/image-classification-api-transfer-learning/sdnet2018decksamples.png)

## Install NuGet packages

In [1]:
#r "nuget: Microsoft.Azure.CognitiveServices.Vision.CustomVision.Training, 2.1.0-preview"
#r "nuget: Microsoft.Azure.CognitiveServices.Vision.CustomVision.Prediction, 2.0.0"

Installed Packages Microsoft.Azure.CognitiveServices.Vision.CustomVision.Prediction, 2.0.0 Microsoft.Azure.CognitiveServices.Vision.CustomVision.Training, 2.1.0-preview

In [2]:
using Microsoft.Azure.CognitiveServices.Vision.CustomVision.Prediction;
using Microsoft.Azure.CognitiveServices.Vision.CustomVision.Training;
using Microsoft.Azure.CognitiveServices.Vision.CustomVision.Training.Models;
using System.IO;
using System.Threading;

## Get Azure Custom Vision Service credentials

In [3]:
var trainingEndpoint = Environment.GetEnvironmentVariable("VISION_TRAINING_ENDPOINT");
var trainingKey = Environment.GetEnvironmentVariable("VISION_TRAINING_KEY");
var predictionEndpoint = Environment.GetEnvironmentVariable("VISION_PREDICTION_ENDPOINT");
var predictionKey = Environment.GetEnvironmentVariable("VISION_PREDICTION_KEY");
var predictionResourceId = Environment.GetEnvironmentVariable("VISION_PREDICTION_RESOURCE_ID");

## Utility Functions

In [4]:
async Task<bool> ProjectExistsAsync(CustomVisionTrainingClient client, string projectName)
{
    var projects = await client.GetProjectsAsync();
    return projects.Where(x => x.Name == projectName).Count() > 0 ? true : false;
}

async Task<bool> TagExistsAsync(CustomVisionTrainingClient client, string projectName, string tagName)
{
    var projects = await client.GetProjectsAsync();
    var project = projects.First(x => x.Name == projectName);
    var tags = await client.GetTagsAsync(project.Id);
    return tags.Where(x => x.Name == tagName).Count() > 0 ? true : false;
}

## Configure Azure Custom Vision training client

In [5]:
var trainingCredentials = new Microsoft.Azure.CognitiveServices.Vision.CustomVision.Training.ApiKeyServiceClientCredentials(trainingKey);
var trainingClient = new CustomVisionTrainingClient(trainingCredentials)
{
    Endpoint = trainingEndpoint
};

## Create Azure Custom Vision Project

In [6]:
var projectName = "Infrastructure CV Project";
var projectExists = await ProjectExistsAsync(trainingClient, projectName);
Project project;
if(!projectExists)
{
    project = await trainingClient.CreateProjectAsync("Infrastructure CV Project", "An image classifier to detect cracks in pavement");
}
else
{
    var projects = await trainingClient.GetProjectsAsync();
    project = projects.First(x => x.Name == projectName);
}

## Create image tags

- Cracked
- Uncracked

In [7]:
var crackedTagName = "Cracked";
var crackedTagExists = await TagExistsAsync(trainingClient, projectName, crackedTagName);
Tag crackedTag;
if(!crackedTagExists)
{
    crackedTag = await trainingClient.CreateTagAsync(project.Id, crackedTagName);
}
else
{
    var tags = await trainingClient.GetTagsAsync(project.Id);
    crackedTag = tags.First(x => x.Name == crackedTagName);
}


In [8]:
var uncrackedTagName = "Uncracked";
var uncrackedTagExists = await TagExistsAsync(trainingClient, projectName, crackedTagName);
Tag uncrackedTag;
if(!uncrackedTagExists)
{
    uncrackedTag = await trainingClient.CreateTagAsync(project.Id, uncrackedTagName);
}
else
{
    var tags = await trainingClient.GetTagsAsync(project.Id);
    uncrackedTag = tags.First(x => x.Name == crackedTagName);
}

## Upload images

### Define image directory and file paths

In [9]:
var crackedImageDirectory = Path.GetFullPath(Path.Join("..","Data","assets","CD"));
var uncrackedImageDirectory = Path.GetFullPath(Path.Join("..","Data","assets","UD"));

In [10]:
var crackedImagePaths = Directory.GetFiles(crackedImageDirectory);
var uncrackedImagePaths = Directory.GetFiles(uncrackedImageDirectory);

### Upload images

In [11]:
foreach(var image in crackedImagePaths)
{
    using(var stream = new MemoryStream(File.ReadAllBytes(image)))
    {
        trainingClient.CreateImagesFromData(project.Id, stream, new List<Guid>() {crackedTag.Id});
    }
}

In [12]:
foreach(var image in uncrackedImagePaths)
{
    using(var stream = new MemoryStream(File.ReadAllBytes(image)))
    {
        trainingClient.CreateImagesFromData(project.Id, stream, new List<Guid>() {uncrackedTag.Id});
    }
}

## Train custom vision model

In [13]:
var iteration = trainingClient.TrainProject(project.Id);

Console.WriteLine("Training");

while(iteration.Status == "Training")
{
    Console.Write(".");
    Thread.Sleep(5000);
    iteration = trainingClient.GetIteration(project.Id, iteration.Id);
}

Training
........................................................

## Publish trained model

In [14]:
var modelName = "InfrastructureClassifierModel"; 
await trainingClient.PublishIterationAsync(project.Id, iteration.Id, modelName, predictionResourceId);

## Test the model

### Configure Azure Custom Vision prediction client

In [15]:
var predictionCredentials = new Microsoft.Azure.CognitiveServices.Vision.CustomVision.Prediction.ApiKeyServiceClientCredentials(predictionKey);
var predictionClient = new CustomVisionPredictionClient(predictionCredentials)
{
    Endpoint = predictionEndpoint
};

### Get images to test

In [16]:
var crackedImage = crackedImagePaths.First();
var crackedImageStream = new MemoryStream(File.ReadAllBytes(crackedImage));

### Make prediction

In [17]:
var prediction = await predictionClient.ClassifyImageAsync(project.Id, modelName, crackedImageStream);

### Display prediction results

In [18]:
prediction

Microsoft.Azure.CognitiveServices.Vision.CustomVision.Prediction.Models.ImagePrediction Id 752574b2-e72b-45bc-81cc-0892f69c4986 Project 72c48d1f-b3d2-4474-9867-e531be8c5afc Iteration cce703c3-0bd1-41ff-ab2d-b6ed8e7b64b0 Created 2023-07-25 22:09:56Z Predictions index value 0 Microsoft.Azure.CognitiveServices.Vision.CustomVision.Prediction.Models.PredictionModel Probability 1 TagId afea6a44-1977-4365-b1ad-21bf4429b3c1 TagName Cracked BoundingBox <null> TagType Regular